# Introduction
N-gram models are a simple and effective method for predicting the next item in a sequence. An n-gram is a contiguous sequence of n items from a given sample of text or speech. The items can be phonemes, syllables, letters, words, or base pairs according to the application. N-gram models are widely used in natural language processing (NLP) tasks such as text prediction, spell checking, and speech recognition. They work by calculating the probability of each word in a sentence based on the occurrence of its preceding words.

# Mathematical Background
The probability of a word sequence in an n-gram model is given by the conditional probability of each word given the previous words. For a bigram model (2-gram), the probability of the word sequence "w1 w2 w3" is calculated as P(w3|w2) * P(w2|w1) * P(w1). Smoothing techniques, like Laplace smoothing, are used to handle cases where the probability of a word sequence is zero.

# Implementation

In [1]:
# Ensure NLTK is installed:
# !pip install nltk

import nltk
from nltk.util import ngrams
from collections import defaultdict
from nltk.tokenize import word_tokenize
import numpy as np
import nltk.corpus
import random as rnd
from collections import defaultdict
from tqdm import tqdm
import random
nltk.download('punkt')
nltk.download('brown')

[nltk_data] Downloading package punkt to /home/michal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /home/michal/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [2]:
# Data Preparation
# sample_text is converted to a list of strings
# Feel free to try different texts!
sample_text = "This is a sample text for n-gram modeling. N-gram models are useful and n-gram models are useful."
tokens = sample_text.lower().split()
# remove periods
tokens = [token.replace(".", "") for token in tokens]
tokens

['this',
 'is',
 'a',
 'sample',
 'text',
 'for',
 'n-gram',
 'modeling',
 'n-gram',
 'models',
 'are',
 'useful',
 'and',
 'n-gram',
 'models',
 'are',
 'useful']

In [3]:
# Building the n-gram model
# Takes n as the number of consecutive words to be considered
# Keeps track of the bigrams as well as their occurence
def n_gram_model(n, sentences):
    n_grams = defaultdict(int)
    for sentence in tqdm(sentences): #tqdm shows progress bar
        words = [word for word in sentence]
        for i in range(len(words) - n + 1):
            n_grams[tuple(words[i:i + n])] += 1
    return n_grams

In [4]:
# Note that the function expects a list of sentences. In this case it is a list of 1 sentence
bigram_model = n_gram_model(2, [tokens]) 

100%|██████████| 1/1 [00:00<00:00, 4733.98it/s]


In [5]:
bigram_model

defaultdict(int,
            {('this', 'is'): 1,
             ('is', 'a'): 1,
             ('a', 'sample'): 1,
             ('sample', 'text'): 1,
             ('text', 'for'): 1,
             ('for', 'n-gram'): 1,
             ('n-gram', 'modeling'): 1,
             ('modeling', 'n-gram'): 1,
             ('n-gram', 'models'): 2,
             ('models', 'are'): 2,
             ('are', 'useful'): 2,
             ('useful', 'and'): 1,
             ('and', 'n-gram'): 1})

# Easy Example: Building a Simple Text Predictor

In [6]:
def predict_next_word(n_gram_model, word):
    word = tuple([word])
    candidates = [(key[-1], count) for key, count in n_gram_model.items() if key[:-1] == word] # go till the last last but one
    if candidates:
        return max(candidates, key=lambda x: x[1])[0]
    else:
        return None

In [7]:
# Test the predictor
print(predict_next_word(bigram_model, 'is'))

a


# Example: Building and Evaluating a Trigram Model
Objective:
Implement a trigram (3-gram) model using the Brown Corpus, and then use this model to generate new sentences. Finally, evaluate its performance by calculating the perplexity of a test set.

### Part 1: Building the Trigram Model
You should first retrieve the data and split it into training and testing. You may choose to implement preprocessing steps such as converting to lowercase or removing punctuation. Then, build your model.

In [8]:
# (Down)load the Brown Corpus
import nltk
nltk.download('brown')
nltk.download('punkt')  # For sentence tokenization
from nltk.corpus import brown

[nltk_data] Downloading package brown to /home/michal/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/michal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# Get the data from the brown corpus
sentences = nltk.corpus.brown.sents()

In [10]:
import re

# Optional preprocessing examples
# Make sure to use the appropriate variable names if you choose to use these

# Convert to lowercase:
lowercase_sentences = [[word.lower() for word in sentence] for sentence in sentences]
print(lowercase_sentences[0])

# Remove punctuation with regular expression
pattern = re.compile(r'[^\w\s]')

# Convert sentences to lowercase and remove punctuation
cleaned_sentences = [[pattern.sub('', word) for word in sentence] for sentence in lowercase_sentences]
print(cleaned_sentences[0])

['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', 'atlantas', 'recent', 'primary', 'election', 'produced', '', 'no', 'evidence', '', 'that', 'any', 'irregularities', 'took', 'place', '']


In [11]:
# Add start and end tokens to each sentence
# There should be n-1 start tokens
# We will be working with trigrams, so two start tokens are needed
sentences_with_tokens = [['<start>'] + ['<start>'] + sentence + ['<end>'] for sentence in sentences]

In [12]:
# Shuffle and split the data
import random
# Shuffle the sentences
random.seed(42)  # For reproducibility
random.shuffle(sentences_with_tokens)

# Split the data (80% train, 20% test)
split_idx = int(len(sentences_with_tokens) * 0.8)
train_sentences = sentences_with_tokens[:split_idx]
test_sentences = sentences_with_tokens[split_idx:]

In [13]:
# Build the model
n = 3
trigram_model = n_gram_model(n, train_sentences) 

100%|██████████| 45872/45872 [00:00<00:00, 67591.96it/s]


### Part 2: Generating New Sentences
Use your trigram model to generate a sentence. Start with a seed of two words and then use the model to find the next word with the highest count. Append this word to the sentence, and then use the last two words of the sentence as the new seed. Repeat this process until you generate an end token (\</s>).

In [14]:
def generate_sentence(n, n_grams, seed=None, n_words=10):
    sentence = []
    
    # Start the sentence with <start> tokens
    for _ in range(n-1):
        sentence.append("<start>")

    for i in range(n_words):
        if i == 0 and seed != None:
            sentence.extend(seed) # add seed to sentence
        else:
            last_n_minus_1_words = tuple(sentence[-(n-1):])  # Consider the last (n-1) words
            if "<end>" in last_n_minus_1_words:  # Stop generating if <end> token encountered
                break
            
            next_word_candidates = [key[-1] for key in n_grams if key[:-1] == last_n_minus_1_words]
            if next_word_candidates: # Check if the previous word is in the model 
                next_word_counts = [n_grams[key] for key in n_grams if key[:-1] == last_n_minus_1_words]
                next_word = next_word_candidates[next_word_counts.index(max(next_word_counts))]
                sentence.append(next_word)
            else:
                break  # If no matching n-grams, stop generating the sentence

    return ' '.join(sentence)

In [15]:
generate_sentence(n, trigram_model, seed=["This", "is"], n_words=50)

'<start> <start> This is a good deal of pleasure ; ; <end>'

The generate_sentence() function chooses the word with the highest count
You may want to implement the function such that it turns the counts into probabilities instead
i.e. the higher the count, the higher the probability of selecting the word.

### Part 3: Evaluating the Model (Perplexity Calculation)
Perplexity is a common metric to evaluate language models. This implements a function to calculate the perplexity of the trigram model on a test set.

In [17]:
import math

def compute_perplexity(n, test_data, n_grams):
    total_log_prob = 0
    total_words = 0

    for sentence in tqdm(test_data):
        for i in range(n - 1, len(sentence)):
            full_context = tuple(sentence[i - n + 1:i + 1]) # Context + current word
            context = tuple(sentence[i - n + 1:i])  # Context for the current word

            # Calculate the probability of the current word given the context
            if n_grams[full_context]: # Check if the sequence of n words exists in the model
                next_word_counts = [n_grams[key] for key in n_grams if key[:-1] == context]
                curr_word_prob = n_grams[full_context] / sum(next_word_counts)                
                total_log_prob += math.log(curr_word_prob)
            else: # if the curr_word was not encountered, assign a small probability
                prob = 1e-6 # can be adjusted based on your dataset
                total_log_prob += prob
            
            total_words += 1

    avg_log_prob = total_log_prob / total_words
    perplexity = math.exp(-avg_log_prob)
    return perplexity

In [ ]:
# Using a subset of the test set as this runs pretty slowly
print("Perplexity:", compute_perplexity(n, random.sample(test_sentences, 100), trigram_model))